# Create report
This is a notebook to load all assets into data frame and export it to Google sheets.

It works with
* Stocks
* ETFs
* Crypto

It collects holding data from
* Alpaca
* Interactive Broker
* Binance

It collects financial data from
* Yahoo Finance
* Google Finance (Spreadsheets)
* CoinMarketCap

## Common methods
* Definition of Constants
* Exporting to Google Sheets
* Generalisation to load financial data for different assets

In [ ]:
COL_WORKSHEET_STOCK = "stock"
COL_WORKSHEET_ETF = "etf"
COL_WORKSHEET_CRYPTO = "crypto"

COL_TICKER = "ticker"

COL_PRICE_INIT = 'avg_price' # average init purchase price of a single asset (share, etf,...)
COL_PRICE_INIT_USD = 'avg_price_usd' # average init purchase price a single asset (share, etf,...)
COL_PRICE = 'price' # current price of a single asset (share, etf,...)
COL_PRICE_USD = 'price_usd' # current price of a single asset (share, etf,...)

COL_AMOUNT = 'amount'
COL_YIELD = 'yield' # we generalize all passive income(dividends, interests, staking) to yield
COL_YIELD_USD = 'yield_usd' # we generalize all passive income(dividends, interests, staking) to yield
COL_TOT_VALUE = "value" # use the same column name for all assets
COL_TOT_INIT_VALUE = "init_val" # use the same column name for all assets
COL_TOT_YIELD = "Yield" # name of the column that multiplies yield with amount
COL_LIVE_DATA = "live"
COL_CHG_UPDATE = "chg_since_last_update"
COL_TARGET_PRICE = "Target"
COL_PROJ_GAIN = "Proj. Gains"
COL_RETURN = "Return"
COL_PAST_GAIN = "Past Gains"
COL_PORTFOLIO = "portfolio"

COL_BROKER = 'broker'
COL_EXCHANGE = 'exchange'

COL_ASSET_TYPE = 'asset_type'
COL_GOOGLE = "google"
COL_YAHOO = "yahoo"

In [ ]:
import pandas as pd
import os

from dotenv import load_dotenv
load_dotenv()
GSHEET_FILE_NAME = os.getenv("gsheet.name")

def get_spreadsheet():
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(GSHEET_FILE_NAME, scope)
    client = gspread.authorize(creds)
    secret = os.getenv("doc.google.key")
    return client.open_by_key(secret)

def export_df_to_sheet(worksheet_name, df):
    worksheet = get_spreadsheet().worksheet(worksheet_name)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

def get_char_by_col_pos(col_pos):
    return chr(ord('@')+col_pos)

def add_live_data(worksheet_name, tickers, all_columns, currencies = [], crypto = False):
    worksheet = get_spreadsheet().worksheet(worksheet_name)
    pos_col_amount = all_columns.index(COL_AMOUNT)+1
    pos_col_tot_val = all_columns.index(COL_TOT_VALUE)+1
    pos_col_live = len(all_columns) + 1

    # update column headers
    worksheet.update([[COL_LIVE_DATA, COL_CHG_UPDATE]], get_char_by_col_pos(pos_col_live) + "1", raw=False, major_dimension = "ROWS")

    col_amount_character = get_char_by_col_pos(pos_col_amount)
    col_tot_val_character = get_char_by_col_pos(pos_col_tot_val)
    col_live_character = get_char_by_col_pos(pos_col_live)
    col_update_character = get_char_by_col_pos(pos_col_live+1)

    update_cols = []
    counter = 0
    for ticker in tickers:
        if crypto:
            update_cols.append(
                f'=IFERROR(REGEXREPLACE('
                f'IMPORTDATA("https://cryptoprices.cc/{ticker}");'
                f'"[.]"; ","'
                f')*{col_amount_character}{counter+2};'
                f'{col_tot_val_character}{counter+2})'
            )
        else:
            append_str = (f'=GOOGLEFINANCE("{ticker}")*'
                          f'{col_amount_character}'
                          f'{counter+2}')
            if len(currencies) > 0:
                target_currency = currencies[counter]
                if target_currency != "USD":
                    append_str = append_str + f'*GoogleFinance("CURRENCY:{target_currency}USD")'

            update_cols.append(append_str)
        counter += 1
    worksheet.update([update_cols],
                     col_live_character + "2",
                     raw=False,
                     major_dimension = "COLUMNS" )

    update_cols = []
    for counter in range(0, len(tickers)):
        update_cols.append(f'={col_live_character}{counter+2}-{col_tot_val_character}{counter+2}')

    worksheet.update([update_cols], col_update_character + "2", raw=False, major_dimension = "COLUMNS" )


In [ ]:
def calculating_price_data(df, col_price = "currentPrice", col_yield = "dividendRate", percentage = False):
    df[COL_PRICE_INIT_USD] = df.apply(convert, column_name=COL_PRICE_INIT, axis=1)
    df[COL_PRICE_USD] = df.apply(convert, column_name=col_price, axis=1)
    df[COL_YIELD_USD] = df.apply(convert, column_name=col_yield, axis=1).infer_objects().fillna({COL_YIELD_USD:0})

    df[COL_TOT_INIT_VALUE] = (df[COL_AMOUNT] * df[COL_PRICE_INIT_USD]).round(2)
    df[COL_TOT_VALUE] = (df[COL_AMOUNT] * df[COL_PRICE_USD]).round(2)
    if percentage:
        df[COL_TOT_YIELD] = df[COL_YIELD_USD] * df[COL_TOT_VALUE]
    else:
        df[COL_TOT_YIELD] = df[COL_AMOUNT] * df[COL_YIELD_USD]
    return df

## Load Data on Positions from Brokers, Exchanges, Database

* Alpaca
* Interactive Broker (Remember: TWS needs to be started for interactive broker)
* Binance
* SQline


In [ ]:
from ib_insync import *
from requests import Session
ALPACA_KEY = os.getenv("broker.alpaca.key")
ALPACA_SECRET = os.getenv("broker.alpaca.secret")

def load_alpaca_balances():
    from alpaca.trading.client import TradingClient
    trading_client = TradingClient(ALPACA_KEY, ALPACA_SECRET, paper=False)
    print(f"Alpaca Cash: ${trading_client.get_account().cash}")
    positions_data = [
        [position.symbol, position.qty,
         position.avg_entry_price,
         position.exchange.value, 'ALPACA']
        for position in trading_client.get_all_positions()
    ]
    return pd.DataFrame(
        positions_data,
        columns=[COL_TICKER, COL_AMOUNT, COL_PRICE_INIT,
                 COL_EXCHANGE, COL_BROKER])

def load_ib_balances():
    util.startLoop()
    ib =  IB()
    ib.connect()
    av = ib.accountValues()
    account_info = util.df(av)
    print(account_info.loc[account_info.tag == "CashBalance"])
    ibr = util.df(ib.positions())
    ibr[COL_TICKER] = (ibr.contract.apply
                       (lambda x: x.symbol))
    ibr["exchange"] = (ibr.contract.apply
                       (lambda x: x.exchange))
    ibr["broker"] = "INTERACTIVE_BROKER"

    return (ibr[[COL_TICKER, 'position', 'avgCost',
                COL_EXCHANGE, COL_BROKER]].
            rename(columns={'position': COL_AMOUNT,
                            'avgCost': COL_PRICE_INIT}))

def load_binance_balances():
    from binance.client import Client
    binance_client = Client(os.getenv("exchange.binance.api"),
                            os.getenv("exchange.binance.secret"))

    locked_assets = pd.DataFrame(
        [
            (pos["asset"], pos["amount"], pos["apy"])
            for pos in binance_client
        .get_simple_earn_locked_product_position()['rows']],
        columns=[COL_TICKER, COL_AMOUNT, COL_YIELD]
    )
    flexible_assets = pd.DataFrame(
        [(pos["asset"], pos["totalAmount"],
          pos["latestAnnualPercentageRate"])
         for pos in binance_client
         .get_simple_earn_flexible_product_position()['rows']],
        columns=[COL_TICKER, COL_AMOUNT, COL_YIELD])

    assets = pd.concat([locked_assets,
                        flexible_assets])
    assets = assets.astype({COL_AMOUNT: 'float',
                            COL_YIELD: 'float',
                            COL_TICKER: 'string'})
    assets.set_index(COL_TICKER, inplace=True)
    return assets
    return assets


In [ ]:
alpaca_balances = load_alpaca_balances()
IB_balances = load_ib_balances()
binance_balances = load_binance_balances()

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///portfolio.sqlite')
asset_lookup = pd.read_sql('asset_lookup',
                           engine,
                           index_col=COL_TICKER)

all_offline_assets = pd.read_sql('offline_asset', engine, index_col=COL_TICKER)
offline_assets = all_offline_assets[[COL_AMOUNT, COL_PRICE_INIT, COL_EXCHANGE]]
offline_assets.reset_index(drop=False, inplace=True)

## Join Data sources
* Data sources from all provider into one unified structure
* adding lookup data
* adding offline data

In [ ]:
all_positions_raw = pd.concat([alpaca_balances, IB_balances, offline_assets])
all_positions = all_positions_raw.merge(asset_lookup, on=COL_TICKER)

# convert to floats
all_positions[COL_AMOUNT] = all_positions[COL_AMOUNT].astype(float)
all_positions[COL_PRICE_INIT] = all_positions[COL_PRICE_INIT].astype(float).round(2)

# fill n/a values with ticker data
all_positions[COL_YAHOO] = (all_positions[COL_YAHOO].
                            fillna(
    all_positions[COL_TICKER]))
all_positions[COL_GOOGLE] = (all_positions[COL_GOOGLE].
                             fillna(
    all_positions[COL_TICKER]))

In [ ]:
## check if there are positions that are not associated with portfolios
all_tickers = all_positions_raw[COL_TICKER].tolist()
all_tickers_in_lookup_db = asset_lookup.reset_index()[COL_TICKER].tolist()
missing_tickers = list(set(all_tickers) - set(all_tickers_in_lookup_db))
if len(missing_tickers) > 0:
    print("The following positions will not be available as you do not have added them to a portfolio:")
    print(missing_tickers)

## Methods to Enrich data from holdings
* Load data from Yahoo
* Conversion into a unified currency
* Loading financial data from Coinmarketcap

In [ ]:
import yfinance as yf
import pandas as pd

def collect_fin_data(tickers):
    return {ticker: yf.Ticker(ticker).info for ticker in tickers}

def merge_fin_data(df_orig, ticker_data, metrics):
    df = df_orig.copy(deep=True)
    for m in metrics:
        df[m] = None

    for ind in df.index:
        ticker_symbol = df.loc[ind, COL_YAHOO]
        company = ticker_data.get(ticker_symbol, {})

        for m in metrics:
            df.loc[ind, m] = company.get(m, None)

    return df

from currency_converter import CurrencyConverter

def get_conversion(target_currency = "USD"):
    c = CurrencyConverter()
    mapping_exchange_currency = {
        '': c.convert(1, 'USD', target_currency),
        'ARCA': c.convert(1, 'USD', target_currency),
        'NASDAQ': c.convert(1, 'USD', target_currency),
        'NYSE': c.convert(1, 'USD', target_currency),
        'BATS': c.convert(1, 'USD', target_currency),
        'PINK': c.convert(1, 'USD', target_currency),
        'IBIS': c.convert(1, 'EUR', target_currency),
        'IBIS2': c.convert(1, 'EUR', target_currency),
        'AEB': c.convert(1, 'EUR', target_currency),
        'VSE': c.convert(1, 'EUR', target_currency),
        'AMEX': c.convert(1, 'USD', target_currency),
        'BVME': c.convert(1, 'EUR', target_currency),
        'STU': c.convert(1, 'EUR', target_currency),
        'SBF': c.convert(1, 'EUR', target_currency),
        'LSE': c.convert(1, 'GBP', target_currency) / 100, # LSE uses GBX (penny sterling)
        'AMX': 0.0026, # Armenian Dram not supported
        'EBS': c.convert(1, 'CHF', target_currency),
        'CPH': c.convert(1, 'DKK', target_currency),
        'PRA': c.convert(1, 'CZK', target_currency)
    }
    return mapping_exchange_currency
conversion = get_conversion()

def convert(row, column_name):
    return (row[column_name] *
            conversion[row[COL_EXCHANGE]])

def add_coinmarketcap_prices(df):
    session = Session()
    session.headers.update({
        'Accepts' : 'application/json',
        'X-CMC_PRO_API_KEY': os.getenv("broker.coinmarketcap.com")
    })
    res = session.get(
        'https://pro-api.coinmarketcap.com/v2/'
        'cryptocurrency/quotes/latest',
        params={
            'symbol': ','.join(df.index.unique()),
            'convert': 'USD'
        }
    )

    if res.status_code != 200:
        print(f"API Error: {res.status_code} - {res.text}")
        return df

    data = res.json().get("data", {})
    prices = {
        coin: data[coin][0]['quote']['USD']['price']
        for coin in data
        if 'USD' in data[coin][0]['quote']
    }
    df[COL_PRICE] = df.index.map(prices)

    return df.astype({COL_PRICE: 'float'})

# Stocks

In [ ]:
shares = all_positions[all_positions[COL_ASSET_TYPE].isin(["STOCK"])]
unique_stock_tickers = shares[COL_YAHOO].unique().tolist()
yfin_data_stocks = collect_fin_data(unique_stock_tickers)

In [ ]:
#ratios = ["currentPrice", "targetMeanPrice", "dividendRate", "profitMargins", "forwardPE", "fiftyDayAverage", "trailingPegRatio", "forwardPE", "trailingPE"]
ratios = ["currentPrice", "targetMeanPrice", "dividendRate"]
stock_info = merge_fin_data(shares, yfin_data_stocks, ratios)

In [ ]:
stock_info = stock_info.infer_objects().fillna(0)
stock_info = calculating_price_data(stock_info)

# additional aggregations
stock_info[COL_TARGET_PRICE] = stock_info.apply(convert, column_name="targetMeanPrice", axis=1)
stock_info[COL_PAST_GAIN] = stock_info[COL_TOT_VALUE] - stock_info[COL_TOT_INIT_VALUE]
stock_info[COL_PROJ_GAIN] = (stock_info[COL_TARGET_PRICE] - stock_info[COL_PRICE_USD]) * stock_info[COL_AMOUNT]

In [ ]:
# export
export_stock_columns = [COL_TICKER, COL_PORTFOLIO, COL_TOT_VALUE, COL_TOT_YIELD, COL_AMOUNT, COL_PROJ_GAIN, COL_PAST_GAIN]
export_df_to_sheet(COL_WORKSHEET_STOCK, stock_info[export_stock_columns])
add_live_data(COL_WORKSHEET_STOCK, stock_info[COL_GOOGLE].tolist(), export_stock_columns, stock_info["currency"].tolist())

## ETF

In [ ]:
all_etfs_held = all_positions[all_positions[COL_ASSET_TYPE].isin(["ETF"])]
yfin_data_etf = collect_fin_data(all_etfs_held[COL_YAHOO].unique().tolist())
etf_info = merge_fin_data(all_etfs_held, yfin_data_etf, ["navPrice", "yield"])
etf_info = etf_info.infer_objects().fillna(0)
etf_info = calculating_price_data(etf_info, "navPrice", "yield", percentage = True)
export_etf_columns = [COL_TICKER, COL_PORTFOLIO, COL_TOT_VALUE, COL_TOT_YIELD, COL_AMOUNT, COL_TOT_INIT_VALUE]
export_df_to_sheet(COL_WORKSHEET_ETF, etf_info[export_etf_columns])
add_live_data(COL_WORKSHEET_ETF, etf_info[COL_GOOGLE].tolist(), export_etf_columns)

## Bond

In [ ]:
all_bonds_held = all_positions[all_positions[COL_ASSET_TYPE].isin(["BOND"])]
all_bonds_held = all_bonds_held[[COL_TICKER, COL_AMOUNT, COL_PRICE_INIT, COL_EXCHANGE]]
all_bonds_held = all_bonds_held.dropna(axis='rows')
all_bonds_held[COL_TOT_VALUE] = all_bonds_held.apply(convert, column_name=COL_PRICE_INIT, axis=1) * all_bonds_held[COL_AMOUNT]

In [ ]:
# add yield from offline db
bond_rates = all_offline_assets[all_offline_assets[COL_ASSET_TYPE].isin(['BOND'])][["yield"]]
bond_df_merged_rates = all_bonds_held.merge(bond_rates, on=COL_TICKER)
bond_df_merged_rates[COL_TOT_YIELD] = bond_df_merged_rates[COL_TOT_VALUE] * bond_df_merged_rates[COL_YIELD] / 100

In [ ]:
export_df_to_sheet("bonds", bond_df_merged_rates[[COL_TICKER, COL_TOT_VALUE, COL_TOT_YIELD, COL_AMOUNT]])

## Crypto

In [ ]:
ledger_cryptos = all_offline_assets[all_offline_assets[COL_ASSET_TYPE].isin(["CRYPTO"])]
all_coins_with_prices = add_coinmarketcap_prices(pd.concat([ledger_cryptos, binance_balances], ignore_index=False))
all_coins_with_prices.reset_index(drop=False, inplace=True)

In [ ]:
# aggregate
all_coins_with_prices[COL_TOT_VALUE] = all_coins_with_prices[COL_AMOUNT].astype(float).round(2) * all_coins_with_prices[COL_PRICE].astype(float).round(2)
all_coins_with_prices[COL_TOT_YIELD] = all_coins_with_prices[COL_TOT_VALUE] * all_coins_with_prices[COL_YIELD].astype(float).round(2)

In [ ]:
export_crypto_columns = [COL_TICKER, COL_TOT_VALUE, COL_TOT_YIELD, COL_AMOUNT]
export_df_to_sheet(COL_WORKSHEET_CRYPTO, all_coins_with_prices[export_crypto_columns])
add_live_data(COL_WORKSHEET_CRYPTO, all_coins_with_prices[COL_TICKER].tolist(), export_crypto_columns, True)